In [1]:
cd "/home/sage/Documents/works/12. Developments/07. sage/02. IdIso/"

/home/sage/Documents/works/12. Developments/07. sage/02. IdIso


#  <font color="green">Implementation of IdealToIsogeny algorithm</font>

## <font color="green">0. Prime Search</font>

In [3]:
load('primegen.sage')
find_prime(20, 5)

n: 20
n: 21
n: 22
n: 23
n: 24
n: 25
n: 26
n: 27
n: 28
n: 29
n: 30
n: 31
n: 32
n: 33
n: 34
n: 35
n: 36
n: 37
n: 38
n: 39
n: 40
n: 41
n: 42
n: 43
n: 44
n: 45
n: 46
n: 47
n: 48
n: 49
n: 50
n: 51
n: 52
n: 53
n: 54
n: 55
n: 56
n: 57
n: 58
n: 59
n: 60
n: 61
n: 62
n: 63
n: 64
n: 65
41320706725109395619839 is not prime
n: 66
41320706725109395619839 is not prime
n: 67
41320706725109395619839 is not prime
n: 68
41320706725109395619839 is not prime
n: 69
41320706725109395619839 is not prime
n: 70
41320706725109395619839 is not prime
n: 71
n: 72
n: 73
n: 74
n: 75
4344577164240073596600319 is not prime
n: 76
n: 77
n: 78
n: 79
142048783804718928027975679 is not prime
n: 80
n: 81
n: 82
n: 83
n: 84
n: 85
n: 86
n: 87
n: 88
n: 89
n: 90
n: 91
n: 92
n: 93
n: 94
n: 95
n: 96
n: 97
3588243480271031849611605510717439 is not prime
n: 98
n: 99
n: 100
n: 101
n: 102
n: 103
n: 104
n: 105
n: 106
n: 107
n: 108
2576677316047908210658258803375472639 is not prime
n: 109
2576677316047908210658258803375472639 is not prim

KeyboardInterrupt: 

In [4]:
prime = 393108616061618412489735027422936257282820381081599
Fp = GF(prime)
Fp2.<Fp_i> = GF(prime^2, modulus = x^2 + 1, impl='pari_ffelt')
R.<x> = PolynomialRing(Fp)
print((x^2 + 1).factor())
# Fp4.<Fp_j> = GF(prime^4, modulus = x^4 - 6*x^2 + 7)


KeyboardInterrupt: 

## <font color="green">1. Prerequisites</font>

### Load required libraries

In [2]:
load('quaternion_tools.sage')
load('isogeny.sage')
load('richelot.sage')
load('const.sage')

### Cornaccia algorithm

In [3]:
def cornacchia(d, m):
    try: 
        (x,y) = DiagonalQuadraticForm(QQ, [1,d]).solve(m)
        if not x.is_integer() or not y.is_integer() : raise Exception("No solution")
        return x, y
    except: raise Exception("No solution")

## <font color="green">2. Setting the lab environment</font>

### 1. Get a random ideal $I$ of norm $\ell^e$

In [4]:
I = GetRandomIdealElle()
assert(I.norm() == ell^6)

### 2. Extract the first ideal factor $I_1$
+ $I = I_1 \cdot I_2 \cdot ... \cdot I_k$ and $n(I_i) = \ell$

In [5]:
ellO0 = QA.ideal(O0.basis()).scale(ell)
I1 = AddIdeal(I, ellO0)

### 3. Compute corresponding isogeny $\phi_1$ to $I_1$
+ $ker(\phi_1) = ker(I_1)$

In [207]:
phi1_kernel_generator = left_O0_ideal_to_isogeny_kernel(I1)
phi1 = Isogeny(Fp(0), phi1_kernel_generator)
assert(I1.norm() == ell)

### 4. Get the codomain curve $E_1$ of $\phi_1$ and the dual isogeny $\hat{\phi}_1$

In [7]:
E1 = phi1.codomain_curve
phi1_dual = phi1.dual_isogeny()

### 5. Compute the right order $\mathcal{O}_1$ of $I_1$
+ $\mathcal{O}_1 \cong End(E_1)$

In [8]:
O1 = I1.right_order()

### 6. Extract the second ideal factor $I_2$
+ $\bar{I_1} I \cap \ell \mathcal{O}_1 = I_2$

In [9]:
I2 = I1.conjugate() * I
I2 = I2.scale(1/ell)
ellO1 = QA.ideal(O1.basis()).scale(ell)
I2 = AddIdeal(I2, ellO1)

assert(I2.norm() == ell and I2.left_order() == O1)

## <font color="green">3. Experiments</font>

+ Compute the generator $\alpha_2$ of $I_2$ such that $I_2 = \alpha_2 \mathcal{O_1} + \ell \mathcal{O_1}$
+ Hopefully $n(\alpha_2)$ is as small as possible
+ Note that $\alpha_2 \in I_2 \subset \mathcal{O}_1$

In [10]:
I2_generator = ideal_generator(I2)

+ Compute the reduced basis $\{o_{11}, o_{12}, o_{13}, o_{14}\}$ of $\mathcal{O_1}$

In [11]:
O1_basis = GetReducedBasis(O1.basis())

+ Try to find the $\beta \in \mathcal{O}_1$ such that $n(\alpha_2) + n(\beta) = 2^{e_1}\cdot 3^{e_2} | \sqrt{\#E_1(\mathbb{F}_{p^k})}$
+ $\mathrm{ord}_{2}(\#E_1(\mathbb{F}_{p^k})) \approx \mathrm{ord}_2(\#E_1(\mathbb{F}_{p^2}))=2 \times \mathrm{ord}_2(p+1)$ regardless of the extension size $k$
    + So it must be satisfied that $2^e < (p+1)$
+ Find the coefficients $\{c_1, c_2, c_3, c_4\}$ such that $n(\beta) = n(\sum_{i}{c_i o_{1i}}) = 2^e - n(\alpha_2)$

In [12]:
import random
I2_basis = GetReducedBasis(I2.basis())
I2_generator = I2_basis[2]
e1 = 148
e2 = 16
gram = matrix([[1,0,0,0],[0,1,0,0],[0,0,prime,0],[0,0,0,prime]])
M_O1 = matrix([vector(v.coefficient_tuple()) for v in O1_basis])
G = M_O1 * gram * M_O1.transpose()

x,y = 0, 0
k = 100
for i in range(k^2):
    I2_gen = I2_generator + random.randint(-k, k) * I2_basis[1] + random.randint(-k, k) * I2_basis[0]
    if gcd(I2_gen.reduced_norm() ,6) != 1: continue
    target = 2^e1*3^e2 - I2_gen.reduced_norm()
    if kronecker(-G[1][1], target) != 1: continue
    try:
        x,y = cornacchia(G[1][1], target)
        break
    except:
        print("no solution of cornacchia algorithm")
beta = O1_basis[0]*x + O1_basis[1]*y
print((beta.reduced_norm() + I2_gen.reduced_norm()).factor())

no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
2^148 * 3^16


In [28]:
N = 2^e1*3^e2
N_alpha = I2_gen.reduced_norm()
N_beta = beta.reduced_norm()
x_value = Integer(mod(N_beta, N)^-1)

In [29]:
P1, Q1 = phi1.codomain_curve.gens()
cofactor = Integer((prime^2 - 1) / N)
P1, Q1 = cofactor*P1, cofactor*Q1
E1.is_on_curve(P1[0], P1[1])

True

In [209]:
cofactor = phi1.domain_P.order() // 2^a * 3^b

In [210]:
load('isogeny.sage')
phi1_dual = phi1.dual_isogeny()
S1 = cofactor * phi1_dual.domain_P
S2 = cofactor * phi1_dual.domain_Q
T1 = phi1.eval(phi1_dual.eval(S1))
T2 = phi1.eval(phi1_dual.eval(S2))
print(T1)
print(T2)
print(5 * S1)
print(5 * S2)

kernel of dual : 5
(230818177896927689752906222682589608519011933520962*Fp2_i^2 + 58261165302924349313476372306358281824413665762074 : 274604664331960353128542157097415870688693698161552*Fp2_i^2 + 312280919428692022740855517647998549777024637128423 : 1)
(380387758288221824709748275287394593890029479302450*Fp2_i^2 + 76056319009346496374629589174082649481252827445973 : 86047929183380062341286262205191691409027077547008*Fp2_i^2 + 330305734112286972633621142943932580358032516340420 : 1)
(230818177896927689752906222682589608519011933520962*Fp2_i^2 + 58261165302924349313476372306358281824413665762074 : 118503951729658059361192870325520386594126682920047*Fp2_i^2 + 80827696632926389748879509774937707505795743953176 : 1)
(380387758288221824709748275287394593890029479302450*Fp2_i^2 + 76056319009346496374629589174082649481252827445973 : 307060686878238350148448765217744565873793303534591*Fp2_i^2 + 62802881949331439856113884479003676924787864741179 : 1)


In [214]:
load('isogeny.sage')
psi_P1 = eval_endomorphism(x_value * I2_gen * beta, P1, phi1, phi1_dual)
psi_Q1 = eval_endomorphism(x_value * I2_gen * beta, Q1, phi1, phi1_dual)

(10989476619759778982924609042373344595959605908935277068431540552651433939612697815, -7037109178359518211001042402852319500342514067189893161279333351257904671643979975, -778260731133292280393124102769389623374282734982317129606757866277128265113578, 1468556522855048243294335346793559220835273900702731197064268290913761235860825)
(10989476619759778982924609042373344595959605908935277068431540552651433939612697815, -7037109178359518211001042402852319500342514067189893161279333351257904671643979975, -778260731133292280393124102769389623374282734982317129606757866277128265113578, 1468556522855048243294335346793559220835273900702731197064268290913761235860825)


In [215]:
print((x_value^2 * I2_gen.reduced_norm() * beta.reduced_norm() + 1) % N)

0


In [267]:
points = [P1, Q1, psi_P1, psi_Q1]
points = [E1((e[0], e[1])) for e in points]
print(points[0].weil_pairing(points[1], N)*points[2].weil_pairing(points[3], N))
ell_cofactor = sqrt(phi1.codomain_curve.cardinality()) // ell
[ell_P, ell_Q] = [point * ell_cofactor for point in E1.gens()]
points = points + [ell_P, ell_Q]


1


In [268]:
load('../01. jacobian_isogeny/22_isogeny.sage')
points = [(points[0], points[2]), (points[1], points[3]), (points[4], E1(0)), (points[5], E1(0))]
a, b = 148, 16
kernel2 = [(3^b*2^(a-1)*D[0], 3^b*2^(a-1)*D[1]) for D in [points[0], points[1]]]
E_start = E1
h, points, isog = Gluing22(E_start, E_start, kernel2, eval_points = points)

In [269]:
kernel22 = [3^b * 2 * D for D in points[:2]]
j, points = isogeny_22(kernel22, points, a-2)

In [270]:
load('../01. jacobian_isogeny/33_isogeny.sage')
kernel33 = [2 * D for D in points[:2]]
K = Fp
j, points = isogeny_33(kernel33, points, b)

In [271]:
G1 = points[0][0]
G2 = points[1][0]
h = j.curve().hyperelliptic_polynomials()[0]
G3, r3 = h.quo_rem(G1*G2)
assert r3 == 0
delta = Matrix(G.padded_list(3) for G in (G1, G2, G3))
if delta.determinant() : print("error")

In [297]:
load('../01. jacobian_isogeny/22_isogeny.sage')
kernel22 = [D for D in points[:2]]
prodE, eval_points = Splitting22(kernel22, [points[3]])

[(x^2 + (18065267916394543731926063862158489327967804410666*Fp2_i^2 + 388388085322030756851931494109646429252569874440862)*x + 59278634119698150572874437807364224351198570268865*Fp2_i^2 + 94540731328554505813755600664033532749815692190382, y + (23043599952750316162311532774765022334022281593212*Fp2_i^3 + 73419278795707088195948470831667961977326222842942*Fp2_i)*x + 179147668767812230292547490921429272304846376885214*Fp2_i^3 + 354830706196481732433965484864668409177654547390080*Fp2_i)]


In [295]:
eval_points[0][0].order()

5

In [298]:
ell_P

(279600171774720663311298322018651873790476926121808*Fp2_i^2 + 19768045554754498458081483921089373090145545954368 : 314152714514471212888708034329563485841740813926129*Fp2_i^2 + 303918225473453320009834601217245816923524695532871 : 1)

In [299]:
I2_gen.reduced_norm().factor()

5 * 2239 * 573928486816147241 * 611829880742157071393137201

In [264]:
points

[]